In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('../src')
from icgem import download_ggm, read_icgem

In [3]:
model = 'GO_CONS_GCF_2_TIM_R6e'
# model = 'XGM2019e_2159'
download_ggm(model_name=model)

model = 'downloads/' + model

Cannot verify completeness of GO_CONS_GCF_2_TIM_R6e.gfc due to connectivity issues. Using the existing file.



Read GGM

In [4]:
model = model + '.gfc'
shc = read_icgem(model)

In [5]:
for key, value in shc.items():
    if 'Cnm' in key or 'Snm' in key:
        continue
    print(f'{key:10}', value)

a          6378136.46
nmax       300
GM         398600441500000.0
tide_sys   zero_tide
